# Simulating Many Sources in `castor_etc`

Isaac Cheng - April 2022

This notebook shows how to process many point sources using the `castor_etc` Python
package. Note that we do not yet support generating multiple sources in the same aperture
(but you can always simulate your own and [use your own data](./custom_source.ipynb) with
the ETC!).


In [1]:
import astropy.units as u
import numpy as np
import pandas as pd
from IPython.display import display

from castor_etc.background import Background
from castor_etc.photometry import Photometry
from castor_etc.sources import PointSource
from castor_etc.telescope import Telescope

We will use the same `Telescope` and (sky) `Background` objects for all of our photometry
calculations.


In [2]:
MyTelescope = Telescope()
MyBackground = Background()
MyBackground.add_geocoronal_emission("avg")

Now we will generate many different point sources at different AB magnitudes, then find
the time required (in seconds) to achieve a signal-to-noise ratio of 5 and 10 in each
passband.


In [3]:
TARGET_SNRS = [5, 10]
AB_MAGS = np.arange(22, 29, 0.1)
REDDENING = 0.09  # E(B-V) for the LMC

wavelengths = np.arange(90, 1200) * u.nm  # generate the spectrum over these wavelengths

# Make some dictionaries to store the results of our calculations
times_to_one_snr = dict.fromkeys(AB_MAGS)
times_to_reach_snrs = {snr: times_to_one_snr.copy() for snr in TARGET_SNRS}

for ab_mag in AB_MAGS:
    #
    # Initialize a point source and generate a flat spectrum in units of AB magnitude
    #
    MySource = PointSource()
    MySource.generate_uniform(wavelengths=wavelengths, value=ab_mag, unit="ABmag")
    #
    # Initialize a Photometry object and use an optimal aperture for a point source
    #
    MyPhot = Photometry(MyTelescope, MySource, MyBackground)
    MyPhot.use_optimal_aperture(quiet=True)  # don't print encircled energy fraction
    for snr in TARGET_SNRS:
        #
        # Find the time, in seconds, required to reach the specified signal-to-noise ratio
        #
        times_to_reach_snrs[snr][ab_mag] = MyPhot.calc_snr_or_t(
            snr=snr, reddening=REDDENING, quiet=True
        )

In [4]:
for snr in TARGET_SNRS:
    print(f"Times (s) required to reach S/N = {snr}")
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        display(pd.DataFrame(times_to_reach_snrs[snr]).round(2).T)

Times (s) required to reach S/N = 5


,uv,u,g
22.0,9.00,7.44,5.66
22.1,9.87,8.16,6.21
22.2,10.82,8.95,6.82
22.3,11.87,9.82,7.49
22.4,13.01,10.77,8.23
22.5,14.27,11.82,9.04
22.6,15.65,12.96,9.93
22.7,17.17,14.22,10.91
22.8,18.83,15.60,11.99
22.9,20.65,17.12,13.18


Times (s) required to reach S/N = 10


,uv,u,g
22.0,25.64,21.23,16.25
22.1,28.12,23.29,17.85
22.2,30.84,25.55,19.61
22.3,33.82,28.03,21.56
22.4,37.10,30.76,23.70
22.5,40.69,33.75,26.06
22.6,44.63,37.03,28.66
22.7,48.95,40.64,31.54
22.8,53.70,44.60,34.71
22.9,58.90,48.95,38.21
